<a href="https://colab.research.google.com/github/greymattar/ECG_WGAN_AugmentationClassification/blob/main/50_Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/gdrive')
import sys
import os
#sys.path.append('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')
os.chdir('/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main')
print(os.getcwd())

Mounted at /content/gdrive
/content/gdrive/MyDrive/Augmentation-of-ECG-Training-Dataset-with-CGAN-main


In [ ]:
!pip install wfdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
#splitting into training and testing
import json
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from datetime import datetime
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import csv
from torchvision import datasets
import torchvision.transforms as transforms
from collections import Counter
import gc
import copy

from tqdm import tqdm

from ekg_class import dicts
import torch.nn as nn
from models_classifier import EcgResNet34
from sklearn.metrics import classification_report as report
from sklearn.metrics import confusion_matrix as cf_matrix
from utils import print_confusion_matrix

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.now()

print(("\n" + "*" * 61 + "\n\t\t\t\t\tstart time  {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 61).format(
    start_time.hour, start_time.minute, start_time.second))

drive = ""
myPath_base = os.path.join(drive, "")
path_aux = 'paper3_DM\\paper3_data'
myPath_base = os.path.join(myPath_base, '')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyper-parameters etc.
dry_run = False
BATCH_SIZE = 16
LEARNING_RATE = 1e-3
NUM_CLASSES = 9
tr_ts_ratio = 0.9     # "training set / test set" split ratio
len_ratio = 0.5

if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 10
#classes2keep = ['/', 'A', 'L', 'N', 'R', 'f', 'j']
#classes2keep_folder = ['P', 'A', 'L', 'N', 'R', 'f', 'j']

classes2keep = ['N','/', 'A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P', 'A', 'L','R', 'f', 'j','F','a']


#key_aug = "aug"
key_aug = "notaug";

#key_bal = 'balanced'
key_bal = 'imbalanced';

# key_case = 'rl'
# key_case = "02"
key_case = "wgan"

len_ratio = 0.5                   # shorter train sets
#len_ratio = 1                  # shorter train sets
num_samples = 8000
tst_len = 1000
print('\ncase: {}, {}, {}\n'.format(key_case, key_bal, key_aug))
# num_N_samples = int(len_ratio*num_samples)

if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', '')
else:
    len_ratio_str = str(len_ratio)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
path = "gb_dm_case_{}".format(key_case)
myPath_save = os.path.join(myPath_base, path)
os.makedirs(myPath_save, exist_ok=True)

brk = 'here'

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   begin: From whole database select training and testing set   %%%%%%%%%%%%%%%%%%


d_set = "MIT_BIH/tesing"
myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\", d_set,"50classifier_wgan_gp_cl_{}_{}".format(classes2keep_folder,len_ratio))
os.makedirs(myPath_save, exist_ok=True)


# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

with open(os.path.join(myPath_base, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = copy.deepcopy(data2keep_dict)
data_test_dict = copy.deepcopy(data2keep_dict)

for item in data:
    if item[2] in classes2keep:
        data2keep_dict[item[2]].append(item[1])
del data

# randomly splitting the dataset into train and test sets
for key in data2keep_dict.keys():
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

    temp = data_train_dict['/']
    data_train_dict.pop('/')
    data_train_dict['P'] = temp

    temp = data_test_dict['/']
    data_test_dict.pop('/')
    data_test_dict['P'] = temp


# create X, y for train and test sets
X_train = []
y_train = []
'''
for key in data_train_dict.keys():
    X_train.extend(data_train_dict[key])
    idx = [classes2keep_folder.index(key)] * len(data_train_dict[key])
    y_train.extend(idx)

X_test = []
y_test = []
for key in data_test_dict.keys():
    X_test.extend(data_test_dict[key])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test.extend(idx)

y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))
'''

#for shorter training set
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train.extend(idx)

X_test = []
y_test = []
for key in classes2keep_folder:
    X_test.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test.extend(idx)

y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))
a = 0

with open(os.path.join(myPath_save, "X_train.json"), "w") as f:
    json.dump(X_train, f)
with open(os.path.join(myPath_save, "y_train.json"), "w") as f:
    json.dump(y_train, f)
with open(os.path.join(myPath_save, "X_test.json"), "w") as f:
    json.dump(X_test, f)
with open(os.path.join(myPath_save, "y_test.json"), "w") as f:
    json.dump(y_test, f)

'''
path = os.path.join(myPath_base, "X_N_trn.json")
with open(path, 'r') as f:
    X_N_trn = json.load(f)

path = os.path.join(myPath_base, "X_N_tst.json")
with open(path, 'r') as f:
    X_N_tst = json.load(f)

path = os.path.join(myPath_base, "X_L_trn.json")
with open(path, 'r') as f:
    X_L_trn = json.load(f)

path = os.path.join(myPath_base, "X_L_tst.json")
with open(path, 'r') as f:
    X_L_tst = json.load(f)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  end: load X and y   %%%%%%%%%%%%%%%%%%
X_tst = X_N_tst + X_L_tst
y_N_tst = [0 for _ in range(len(X_N_tst))]
y_L_tst = [1 for _ in range(len(X_L_tst))]
y_tst = y_N_tst + y_L_tst

if key_aug == 'aug':
    with open(os.path.join(myPath_base, 'gb_dm_case_{}\\gb_dm_1d_case_{}.json'.format(key_case, key_case)), "r") as f:
        gb_for_aug = json.load(f)

    rand_idx = random.sample(range(0, len(gb_for_aug)), len(gb_for_aug))
    temp = [gb_for_aug[idx] for idx in rand_idx]
    X_aug = temp[:7000-350]
else:
    X_aug = []

# # imbalanced case N: 350, select 350 elements in X_N_trn if 'imbalanced' or 'augmented'
if key_bal == 'imbalanced' or key_aug == 'aug':
    rand_idx = random.sample(range(0, len(X_N_trn)), len(X_N_trn))[:350]
    X_N_trn = [X_N_trn[idx] for idx in rand_idx]

X_N_trn = X_N_trn + X_aug
X_trn = X_N_trn + X_L_trn

y_N_trn = [0 for _ in range(len(X_N_trn))]
y_L_trn = [1 for _ in range(len(X_L_trn))]
y_trn = y_N_trn + y_L_trn
'''
trn_set = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
tst_set = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
trn_loader = DataLoader(trn_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
tst_loader = DataLoader(tst_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

brk = 'here'
# %%%%%%%%%%%%%%%%%     begin save sample plots of classes in classes2keep    %%%%%%%%%%%%%%%%%%%%
'''
for cl in classes2keep:
    fig, axes = plt.subplots(nrows=3, ncols=3)
    fig.suptitle("Class {} ({}: {}), count: {}".
                 format(classes2keep.index(cl), cl, letter2descr[cl], len(data2keep_dict[cl])))

    count = 0
    for i in range(3):
        for j in range(3):
            count += 1
            if count >= len(data2keep_dict[cl]):
                continue
            axes[i][j].plot(data2keep_dict[cl][count])
            axes[i][j].grid()
    plt.savefig(os.path.join(myPath_save, "00_sample_cl_{}.png".format(classes2keep.index(cl))))

plt.close("all")
'''
# %%%%%%%%%%%%%%%%%     end save sample plots of classes in classes2keep      %%%%%%%%%%%%%%%%%%%%


# %%%%%%%%%%%%%%%%%%%% begin:     Select and Initialize Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%
# net = net_cnn(num_classes=NUM_CLASSES).to(device)
# net = net_fc(input_size=INPUT_SIZE, num_classes=NUM_CLASSES).to(device)
net = EcgResNet34(num_classes=NUM_CLASSES).to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
# %%%%%%%%%%%%%%%%%%%% end:     Select and Initialize Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%


NUM_EPOCHS = 10
# %%%%%%%%%%%%%%%%%%%%    begin:  Train Classifier       %%%%%%%%%%%%%%%%%%%%%%%%%%
for epoch in range(NUM_EPOCHS):
    for batch_idx, (inputs, labels) in enumerate(trn_loader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        inputs = inputs.reshape(inputs.shape[0], 1, -1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        scores = net(inputs)
        loss = criterion(scores.squeeze(), labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        # running_loss += loss.item()
        if batch_idx % 200 == 0:  # print every 200 mini-batches
            now = datetime.now()
            print('{:02d}:{:02d}:{:02d}\t\tepoch={:4d} / {:4d}\t\titer={:5d} / {:5d}\t\t\tloss: {:7.5f}'.
                  format(now.hour, now.minute, now.second, epoch, NUM_EPOCHS, batch_idx, len(trn_loader), loss))

print('\n\tFinished Training\n')
# %%%%%%%%%%%%%%%%%%%%    end  Train Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%


print('\tSaving model ...\n')

# %%%%%%%%%%%%%%%%%%%%    begin save model   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
f_name = "classifier_{}_{}_{}.pth".format(key_case, key_bal, key_aug)
PATH = os.path.join(myPath_save, f_name)
torch.save(net.state_dict(), PATH)
# %%%%%%%%%%%%%%%%%%%%    end save model     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%




*************************************************************
					start time  02:08:32
*************************************************************

case: wgan, imbalanced, notaug

train: Counter({0: 33773, 3: 3633, 4: 3266, 1: 3162, 2: 1145, 5: 441, 7: 361, 6: 103, 8: 67})
test: Counter({0: 7505, 3: 807, 4: 725, 1: 702, 2: 254, 5: 98, 7: 80, 6: 22, 8: 14})
02:09:24		epoch=   0 /   10		iter=    0 /  2871			loss: 1.19965
02:09:48		epoch=   0 /   10		iter=  200 /  2871			loss: 3.27782
02:10:11		epoch=   0 /   10		iter=  400 /  2871			loss: 2.37190
02:10:35		epoch=   0 /   10		iter=  600 /  2871			loss: 1.25582
02:10:59		epoch=   0 /   10		iter=  800 /  2871			loss: 0.12806
02:11:23		epoch=   0 /   10		iter= 1000 /  2871			loss: 0.80305
02:11:48		epoch=   0 /   10		iter= 1200 /  2871			loss: 0.97326
02:12:12		epoch=   0 /   10		iter= 1400 /  2871			loss: 0.88709
02:12:36		epoch=   0 /   10		iter= 1600 /  2871			loss: 1.01079
02:13:00		epoch=   0 /   10		iter= 1800 /  2871			loss: 0.343

In [ ]:
# %%%%%%%%%%%%%%%%%%%%    load trained classifier     %%%%%%%%%%%%%%%%%%%%%
path = "gb_dm_case_{}".format(key_case)
myPath = os.path.join(myPath_base, path)
classifier_name = "classifier_{}_{}_{}.pth".format(key_case, key_bal, key_aug)
net.load_state_dict(torch.load(os.path.join(myPath_save, classifier_name)))

brk = 'here'

# %%%%%%%%%%%%%%%%%%    print Classification Report to file      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
trntst = ['tst']
for key_trntst in trntst:
    print('Preparing Classification Report: {}'.format(key_trntst))
    path = os.path.join(myPath_save, 'classification_report_{}_{}_{}_{}.txt'.
                        format(key_trntst, key_case, key_bal, key_aug))
    with open(path, 'w') as sys.stdout:
        y_scores = []
        y_true = []
        y_pred = []
        exec('loader = {}_loader'.format(key_trntst))
        for (inputs, labels) in tqdm(loader):
            inputs = inputs.reshape(inputs.shape[0], 1, -1).to(device)
            labels = labels.to(device)
            temp = net(inputs)
            output = temp.max(dim=1)

            y_true.extend(labels.data.tolist())
            y_pred.extend(output.indices.tolist())
            y_scores.extend((F.softmax(temp, dim=1).tolist()))

        print("\n")
        print("%" * 20 + "\tClassification Report ({} Set, {}, {}, {})\t".
              format(key_trntst, key_case, key_bal, key_aug) + "%" * 20)
        print("Classes:                             {}".format(classes2keep_folder))
        print("Classifier Model:                    {}".format(net._get_name()))
        print(f"number of epochs:                    {NUM_EPOCHS}\n")
        print("Train set length:                    {}".format(len(trn_set)))
        print("Test set length:                     {}".format(len(tst_set)))
        print("Train set reduction length ratio:    {}\t".format(len_ratio_str))
        print('\nClassification Report:')
        print(report(y_true, y_pred, target_names=classes2keep_folder))
        print("\nConfusion Matrix:\n {}".format(cf_matrix(y_true, y_pred)))
        """
        precision, recall, thresholds = precision_recall_curve(y_true, np.asarray(y_scores)[:, 1])
        #here auc function won't work as it works for binary classifier
        pr_recall_auc_score = auc(recall, precision)
        print('\nPrecision-Recall AUC Score:        {:6.4f}'.format(pr_recall_auc_score))
        rocauc_score = roc_auc_score(y_true, np.asarray(y_scores)[:, 1])
        print('\nROC AUC Score:                     {:6.4f}'.format(rocauc_score))

        print_confusion_matrix(cf_matrix(y_true, y_pred), class_names=classes2keep_folder,
                               fig_name="Conf. Matrix_{}_{}_{}_{}".format(key_trntst, key_case, key_bal, key_aug))
        plt.savefig(os.path.join(myPath_save, "cfmx_{}_{}_{}_{}.png".format(key_trntst, key_case, key_bal, key_aug)))
        plt.close("all")


        plt.plot(precision, recall)
        plt.title("Precision - Recall Curve ({}, {}, {}, {})".format(key_trntst, key_case, key_bal, key_aug))
        plt.grid()
        plt.savefig(os.path.join(myPath_save, "prec_recall_curve_{}_{}_{}_{}.png".
                                 format(key_trntst, key_case, key_bal, key_aug)))
        """
        plt.close("all")
        sys.stdout = sys.__stdout__


finish_time = datetime.now()
print(("\n\n\n" + "finish time = {0:02d}:{1:02d}:{2:02.0f}").format(
    finish_time.hour, finish_time.minute, finish_time.second))

laps = finish_time - start_time
tot_sec = laps.total_seconds()
h = int(tot_sec // 3600)
m = int((tot_sec % 3600) // 60)
s = int(tot_sec - (h * 3600 + m * 60))

print("total elapsed time = {:02d}:{:2d}:{:2d}".format(h, m, s))



  0%|          | 2/637 [00:00<00:39, 15.91it/s]

Preparing Classification Report: tst


100%|██████████| 637/637 [00:21<00:00, 29.28it/s]


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

    # Calculate and print the accuracy with desired precision
accuracy = accuracy_score(y_true, y_pred)
accuracy_str = "{:.5f}".format(accuracy)
display("Accuracy: {}".format(accuracy_str))

'Accuracy: 0.97792'

Now 50% Balaned or Augmneted

In [ ]:
#splitting into training and testing
import json
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from datetime import datetime
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import csv
from torchvision import datasets
import torchvision.transforms as transforms
from collections import Counter
import gc
import copy

from tqdm import tqdm

from ekg_class import dicts
import torch.nn as nn
from models_classifier import EcgResNet34
from sklearn.metrics import classification_report as report
from sklearn.metrics import confusion_matrix as cf_matrix
from utils import print_confusion_matrix

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

num2descr, letter2num, letter2descr, num2letter = dicts()
start_time = datetime.now()

print(("\n" + "*" * 61 + "\n\t\t\t\t\tstart time  {0:02d}:{1:02d}:{2:02.0f}\n" + "*" * 61).format(
    start_time.hour, start_time.minute, start_time.second))

drive = ""
myPath_base = os.path.join(drive, "")
path_aux = 'paper3_DM\\paper3_data'
myPath_base = os.path.join(myPath_base, '')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyper-parameters etc.
dry_run = False
BATCH_SIZE = 16
LEARNING_RATE = 1e-3
NUM_CLASSES = 9
tr_ts_ratio = 0.9     # "training set / test set" split ratio
len_ratio = 0.5

if dry_run:
    NUM_EPOCHS = 1
else:
    NUM_EPOCHS = 10
#classes2keep = ['/', 'A', 'L', 'N', 'R', 'f', 'j']
#classes2keep_folder = ['P', 'A', 'L', 'N', 'R', 'f', 'j']

classes2keep = ['N','/', 'A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P', 'A', 'L','R', 'f', 'j','F','a']


key_aug = "aug"
#key_aug = "notaug";

key_bal = 'balanced'
#key_bal = 'imbalanced';

# key_case = 'rl'
# key_case = "02"
key_case = "wgan"

len_ratio = 0.5                  # shorter train sets
#len_ratio = 1                  # shorter train sets
num_samples = 8000
tst_len = 1000
print('\ncase: {}, {}, {}\n'.format(key_case, key_bal, key_aug))
# num_N_samples = int(len_ratio*num_samples)

if '.' in str(len_ratio):
    len_ratio_str = str(len_ratio).replace('.', '')
else:
    len_ratio_str = str(len_ratio)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
path = "gb_dm_case_{}".format(key_case)
myPath_save = os.path.join(myPath_base, path)
os.makedirs(myPath_save, exist_ok=True)

brk = 'here'

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   begin: From whole database select training and testing set   %%%%%%%%%%%%%%%%%%

d_set = "MIT_BIH/tesing"
myPath_save = os.path.join(myPath_base, "PycharmProjects\\paper2_gen_data\\", d_set,"50classifierBalanced_wgan_gp_cl_{}_{}".format(classes2keep_folder,len_ratio))
os.makedirs(myPath_save, exist_ok=True)

# %%%%%%%%%%%%%%%%       begin MIT-BIH Dataset      %%%%%%%%%%%%%%%%%%%%%%%%%%%%

with open(os.path.join(myPath_base, "record_X_y_adapt_win_bef075_aft075_Normalized.json"), "r") as f:
    data = json.load(f)

stats_all_classes = Counter(np.asarray(data, dtype=object)[:, 2])

classes2gb = ['P', 'A', 'L','R', 'f', 'j','F','a']
classes2gb_count=['/', 'A', 'L','R', 'f', 'j','F','a']

class_counts = {class_label: 0 for class_label in classes2gb_count}

# create dictionary of data to be kept
vals = []
for idx in range(len(classes2keep)):
    vals.append([])

data2keep_dict = dict(zip(classes2keep, vals))
data_train_dict = dict(zip(classes2keep_folder, vals))
data_test_dict = dict(zip(classes2keep_folder, vals))
count_N = 0
for item in data:
    if item[2] == 'N'  and count_N < 70000:  # Add condition to limit 'N' data to 10000
        data2keep_dict[item[2]].append(item[1])
        count_N += 1
    elif item[2]!='N' and item[2] in classes2keep:
        data2keep_dict[item[2]].append(item[1])
        class_counts[item[2]] += 1
del data

if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

    temp = class_counts['/']
    class_counts.pop('/')
    class_counts['P'] = temp



max_items_per_class= 5000
with open(os.path.join(myPath_base, "gb_together.json"), "r") as f:
    data_gb = json.load(f)




for item in data_gb:
    if item[1] in classes2gb and class_counts[item[1]] < max_items_per_class:
        data2keep_dict[item[1]].append(item[0])
        class_counts[item[1]] += 1

del data_gb



# randomly splitting the dataset into train and test sets
for key in data2keep_dict.keys():
    val_len = len(data2keep_dict[key])
    idx_train = torch.randperm(val_len)[:int(tr_ts_ratio * val_len)]
    idx_test = torch.randperm(val_len)[int(tr_ts_ratio * val_len)+1:]
    data_train_dict[key] = [data2keep_dict[key][idx] for idx in idx_train]
    data_test_dict[key] = [data2keep_dict[key][idx] for idx in idx_test]
'''
if '/' in classes2keep:
    temp = data2keep_dict['/']
    data2keep_dict.pop('/')
    data2keep_dict['P'] = temp

    temp = data_train_dict['/']
    data_train_dict.pop('/')
    data_train_dict['P'] = temp

    temp = data_test_dict['/']
    data_test_dict.pop('/')
    data_test_dict['P'] = temp

'''
# create X, y for train and test sets
X_train = []
y_train = []
for key in classes2keep_folder:
    temp = data_train_dict[key]
    length = int(len_ratio*len(temp))
    X_train.extend(temp[:length])
    idx = [classes2keep_folder.index(key)] * length
    # print('key {}, idx {}'.format(key, idx))
    y_train.extend(idx)

X_test = []
y_test = []
for key in classes2keep_folder:
    X_test.extend(data_test_dict[key][:])
    idx = [classes2keep_folder.index(key)] * len(data_test_dict[key])
    y_test.extend(idx)

y_test_stat = Counter(y_test)
y_train_stat = Counter(y_train)

print("train: {}".format(y_train_stat))
print("test: {}".format(y_test_stat))

a = 0
with open(os.path.join(myPath_save, "X_train.json"), "w") as f:
    json.dump(X_train, f)
with open(os.path.join(myPath_save, "y_train.json"), "w") as f:
    json.dump(y_train, f)
with open(os.path.join(myPath_save, "X_test.json"), "w") as f:
    json.dump(X_test, f)
with open(os.path.join(myPath_save, "y_test.json"), "w") as f:
    json.dump(y_test, f)

a=0
display("train: {}".format(y_train_stat))
display("test: {}".format(y_test_stat))
trn_set = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
tst_set = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
trn_loader = DataLoader(trn_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
tst_loader = DataLoader(tst_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

brk = 'here'



# %%%%%%%%%%%%%%%%%%%% begin:     Select and Initialize Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%
# net = net_cnn(num_classes=NUM_CLASSES).to(device)
# net = net_fc(input_size=INPUT_SIZE, num_classes=NUM_CLASSES).to(device)
net = EcgResNet34(num_classes=NUM_CLASSES).to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
# %%%%%%%%%%%%%%%%%%%% end:     Select and Initialize Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%


NUM_EPOCHS = 10
# %%%%%%%%%%%%%%%%%%%%    begin:  Train Classifier       %%%%%%%%%%%%%%%%%%%%%%%%%%
for epoch in range(NUM_EPOCHS):
    for batch_idx, (inputs, labels) in enumerate(trn_loader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        inputs = inputs.reshape(inputs.shape[0], 1, -1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        scores = net(inputs)
        loss = criterion(scores.squeeze(), labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        # running_loss += loss.item()
        if batch_idx % 1000 == 0:  # print every 200 mini-batches
            now = datetime.now()
            display('{:02d}:{:02d}:{:02d}\t\tepoch={:4d} / {:4d}\t\titer={:5d} / {:5d}\t\t\tloss: {:7.5f}'.
                  format(now.hour, now.minute, now.second, epoch, NUM_EPOCHS, batch_idx, len(trn_loader), loss))

print('\n\tFinished Training\n')
# %%%%%%%%%%%%%%%%%%%%    end  Train Network       %%%%%%%%%%%%%%%%%%%%%%%%%%%%


display('\tSaving model ...\n')

# %%%%%%%%%%%%%%%%%%%%    begin save model   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
f_name = "classifier_{}_{}_{}.pth".format(key_case, key_bal, key_aug)
PATH = os.path.join(myPath_save, f_name)
torch.save(net.state_dict(), PATH)
# %%%%%%%%%%%%%%%%%%%%    end save model     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%



'train: Counter({0: 31500, 3: 3633, 4: 3266, 1: 3162, 2: 2250, 5: 2250, 6: 2250, 7: 2250, 8: 2250})'

'test: Counter({0: 6999, 3: 807, 4: 725, 1: 702, 2: 499, 5: 499, 6: 499, 7: 499, 8: 499})'

'06:15:18\t\tepoch=   0 /   10\t\titer=    0 /  3300\t\t\tloss: 2.81353'

'06:17:22\t\tepoch=   0 /   10\t\titer= 1000 /  3300\t\t\tloss: 0.95777'

'06:19:27\t\tepoch=   0 /   10\t\titer= 2000 /  3300\t\t\tloss: 0.17833'

...

'07:16:45\t\tepoch=   9 /   10\t\titer=    0 /  3300\t\t\tloss: 0.00575'

'07:18:48\t\tepoch=   9 /   10\t\titer= 1000 /  3300\t\t\tloss: 0.29996'

'07:20:51\t\tepoch=   9 /   10\t\titer= 2000 /  3300\t\t\tloss: 0.06388'

'07:22:55\t\tepoch=   9 /   10\t\titer= 3000 /  3300\t\t\tloss: 0.03062'

'\tSaving model ...\n'

In [ ]:
import json
import os
classes2keep = ['N','/', 'A', 'L', 'R', 'f', 'j','F','a']
classes2keep_folder = ['N','P', 'A', 'L','R', 'f', 'j','F','a']
len_ratio=0.5
# Specify the path where the JSON files are saved
d_set = "MIT_BIH/tesing"
myPath_save = os.path.join( "PycharmProjects\\paper2_gen_data\\", d_set,"50classifierBalanced_wgan_gp_cl_{}_{}".format(classes2keep_folder,len_ratio))
os.makedirs(myPath_save, exist_ok=True)

# Load data into X_train
with open(os.path.join(myPath_save, "X_train.json"), "r") as f:
    X_train = json.load(f)

# Load data into X_test
with open(os.path.join(myPath_save, "X_test.json"), "r") as f:
    X_test = json.load(f)

with open(os.path.join(myPath_save, "y_train.json"), "r") as f:
    y_train = json.load(f)

with open(os.path.join(myPath_save, "y_test.json"), "r") as f:
    y_test = json.load(f)


In [ ]:
# %%%%%%%%%%%%%%%%%%%%    load trained classifier     %%%%%%%%%%%%%%%%%%%%%
path = "gb_dm_case_{}".format(key_case)
myPath = os.path.join(myPath_base, path)
classifier_name = "classifier_{}_{}_{}.pth".format(key_case, key_bal, key_aug)
net.load_state_dict(torch.load(os.path.join(myPath_save, classifier_name)))

brk = 'here'

# %%%%%%%%%%%%%%%%%%    print Classification Report to file      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
trntst = ['tst']
for key_trntst in trntst:
    print('Preparing Classification Report: {}'.format(key_trntst))
    path = os.path.join(myPath_save, 'classification_report_{}_{}_{}_{}.txt'.
                        format(key_trntst, key_case, key_bal, key_aug))
    with open(path, 'w') as sys.stdout:
        y_scores = []
        y_true = []
        y_pred = []
        exec('loader = {}_loader'.format(key_trntst))
        for (inputs, labels) in tqdm(loader):
            inputs = inputs.reshape(inputs.shape[0], 1, -1).to(device)
            labels = labels.to(device)
            temp = net(inputs)
            output = temp.max(dim=1)

            y_true.extend(labels.data.tolist())
            y_pred.extend(output.indices.tolist())
            y_scores.extend((F.softmax(temp, dim=1).tolist()))

        print("\n")
        print("%" * 20 + "\tClassification Report ({} Set, {}, {}, {})\t".
              format(key_trntst, key_case, key_bal, key_aug) + "%" * 20)
        print("Classes:                             {}".format(classes2keep_folder))
        print("Classifier Model:                    {}".format(net._get_name()))
        print(f"number of epochs:                    {NUM_EPOCHS}\n")
        print("Train set length:                    {}".format(len(trn_set)))
        print("Test set length:                     {}".format(len(tst_set)))
        print("Train set reduction length ratio:    {}\t".format(len_ratio_str))
        print('\nClassification Report:')
        print(report(y_true, y_pred, target_names=classes2keep_folder))
        print("\nConfusion Matrix:\n {}".format(cf_matrix(y_true, y_pred)))
        """
        precision, recall, thresholds = precision_recall_curve(y_true, np.asarray(y_scores)[:, 1])
        #here auc function won't work as it works for binary classifier
        pr_recall_auc_score = auc(recall, precision)
        print('\nPrecision-Recall AUC Score:        {:6.4f}'.format(pr_recall_auc_score))
        rocauc_score = roc_auc_score(y_true, np.asarray(y_scores)[:, 1])
        print('\nROC AUC Score:                     {:6.4f}'.format(rocauc_score))

        print_confusion_matrix(cf_matrix(y_true, y_pred), class_names=classes2keep_folder,
                               fig_name="Conf. Matrix_{}_{}_{}_{}".format(key_trntst, key_case, key_bal, key_aug))
        plt.savefig(os.path.join(myPath_save, "cfmx_{}_{}_{}_{}.png".format(key_trntst, key_case, key_bal, key_aug)))
        plt.close("all")


        plt.plot(precision, recall)
        plt.title("Precision - Recall Curve ({}, {}, {}, {})".format(key_trntst, key_case, key_bal, key_aug))
        plt.grid()
        plt.savefig(os.path.join(myPath_save, "prec_recall_curve_{}_{}_{}_{}.png".
                                 format(key_trntst, key_case, key_bal, key_aug)))
        """
        plt.close("all")
        sys.stdout = sys.__stdout__


finish_time = datetime.now()
print(("\n\n\n" + "finish time = {0:02d}:{1:02d}:{2:02.0f}").format(
    finish_time.hour, finish_time.minute, finish_time.second))

laps = finish_time - start_time
tot_sec = laps.total_seconds()
h = int(tot_sec // 3600)
m = int((tot_sec % 3600) // 60)
s = int(tot_sec - (h * 3600 + m * 60))

print("total elapsed time = {:02d}:{:2d}:{:2d}".format(h, m, s))
from sklearn.metrics import accuracy_score


    # Calculate and print the accuracy with desired precision
accuracy = accuracy_score(y_true, y_pred)
accuracy_str = "{:.5f}".format(accuracy)
display("Accuracy: {}".format(accuracy_str))



100%|██████████| 733/733 [00:24<00:00, 29.36it/s]


'Accuracy: 0.97655'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



# Create confusion matrix
cm = confusion_matrix(y_true , y_pred)

classes = ['N', 'P', 'A', 'L', 'R', 'f', 'j', 'F', 'a']


# Calculate percentages
cm_percentages = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Create a pandas DataFrame for the confusion matrix
cm_df = pd.DataFrame(cm_percentages, index=classes, columns=classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Percentages)')
plt.show()


In [ ]:
print("new")